# Query data via Google Calendar API

In [ ]:
import datetime
from pathlib import Path

import pandas as pd

from utils import authenticate_google, query_google_calendar_api

## Authenticate & Subscribe to calendars

In [ ]:
# Authenticate Google Calendar API
oauth2_client_secret_file = 'client_secret_12345....com.json'
scopes = ['https://www.googleapis.com/auth/calendar']

service = authenticate_google(scopes=scopes, oauth2_client_secret_file=oauth2_client_secret_file)

In [ ]:
# Subscribe to selected, shared calendars
new_emails = ["luke.skywalker@rebels.com", "darth.vader@empire.com"]

for email in new_emails:
    service.calendarList().insert(body={'id': email}).execute()

In [ ]:
# Get email-ids of all subscribed calendars
calendars_result = service.calendarList().list().execute()

calendars = calendars_result.get('items', [])
emails = [c['id'] for c in calendars]
print(len(emails))

## Query subscribed calendars

### Basic test query
Just for demonstration

In [ ]:
calendar_results = service.events().list(
   calendarId="luke.skywalker@rebels.com", 
   timeMin='2022-01-01T00:00:00Z',
   timeMax='2022-02-01T00:00:00Z',
   maxResults=1000, 
   singleEvents=True,
   orderBy='startTime').execute()

df_test = pd.DataFrame(calendar_results.get('items', []))
df_test.shape

### Query for all results (pagination) for multiple calendars

In [ ]:
# Define query dates
end_time = datetime.datetime.today()
start_time = datetime.datetime(2020, 1, 1, 0,0,0)
print(start_time, end_time)

In [ ]:
fp_df = "df_calendar.pkl"
if Path(fp_df).exists():
    print("Reading df from saved .pkl file")
    df = pd.read_pickle(fp_df)
else:
    print("No saved .pkl file, querying from Google calendar API")
    df = query_google_calendar_api(service=service, emails=emails[:2], start_time=start_time, end_time=end_time)
    df.to_pickle(fp_df)

df.shape

## Clean & Prepare data

In [ ]:
columns_keep = ["summary", "creator", "start", "end", "attendees", "location", "queried_from", "id"]
df = df[columns_keep]
df = df.rename(columns={"summary": "name"})

df["timeZone"] = df["start"].apply(lambda x : x.get("timeZone", "Europe/Berlin"))
# API delivers entries with dateTime or date, we want a single type in the column
df["start"] = df["start"].apply(lambda x : pd.to_datetime(x.get("dateTime", x.get("date")), utc=True))
df["end"] = df["end"].apply(lambda x : pd.to_datetime(x.get("dateTime", x.get("date")), utc=True))
df["duration"] = df.end - df.start

print(df.shape)
df.head(1)